# Prepare the synthetic test cases

In [ ]:
import copy

import matplotlib.pyplot as plt
import numpy as np

from geone.img import readImageGslib, writeImageGslib, sampleFromImage, pointSetToImage
from geone.img import writePointSetGslib, readPointSetGslib
from geone.imgplot import drawImage2D
from geone.deesseinterface import DeesseInput, deesseRun

In [ ]:
DATA_DIR = 'data/'
SAMPLES_DIR = 'samples/'

In [ ]:
COLOR_SCHEME_BINARY = [ 
        [x/255 for x in [166,206,227]], 
        [x/255 for x in [31,120,180]],
      ]

## Benchmark: training image selection

### Training images

The training images originally apeared in the publication: Pérez, C., Mariethoz, G., Ortiz, J. M., 2014. Verifying the high-order consistency of training images with data for multiple-point geostatistics. Computers & Geosciences 70, 190 – 205. https://doi.org/10.1016/j.cageo.2014.06.001

In [ ]:
def get_and_plot_image(filename):
    image = readImageGslib(filename)
    drawImage2D(image, categ=True, categCol=COLOR_SCHEME_BINARY)
    return image

In [ ]:
ti_A = get_and_plot_image(DATA_DIR+'A.gslib')

In [ ]:
ti_B = get_and_plot_image(DATA_DIR+'B.gslib')

In [ ]:
ti_C = get_and_plot_image(DATA_DIR+'C.gslib')

### Synthetic realities 

In [ ]:
def get_and_plot_DeeSse_simulation(ti):
    # build DeeSse input
    deesse_input = DeesseInput(
        nx=100, ny=100, nz=1,     # dimension of the simulation grid (number of cells)
        sx=1.0, sy=1.0, sz=1.0,   # cells units in the simulation grid (here are the default values)
        ox=0.0, oy=0.0, oz=0.0,   # origin of the simulation grid (here are the default values)
        nv=1, varname='code',     # number of variable(s), name of the variable(s)
        nTI=1, TI=ti,             # number of TI(s), TI (class dsi.Img)
        distanceType=0,           # distance type: proportion of mismatching nodes (categorical var., default)
        nneighboringNode=60,      # max. number of neighbors (for the patterns)
        distanceThreshold=0.05,   # acceptation threshold (for distance between patterns)
        maxScanFraction=0.25,     # max. scanned fraction of the TI (for simulation of each cell)
        npostProcessingPathMax=1, # number of post-processing path(s)
        seed=201912,                 # seed (initialization of the random number generator)
        nrealization=1)       # number of realization(s)
    
    # run the simulation
    image = deesseRun(deesse_input)['sim'][0]
    
    #plot the result
    drawImage2D(image, categ=True, categCol=COLOR_SCHEME_BINARY)
    return image
    

In [ ]:
reality_A = get_and_plot_DeeSse_simulation(ti=ti_A)

In [ ]:
reality_B = get_and_plot_DeeSse_simulation(ti=ti_B)

In [ ]:
reality_C = get_and_plot_DeeSse_simulation(ti=ti_C)

### Sample with different sampling rates

In [ ]:
SAMPLE_SIZES = [25, 50, 75, 100, 125, 150, 175, 200, 400, 1600]
SEED = 201912

In [ ]:
def sample_and_write_gslib(image, size, seed, name):
        sampled_point_set = sampleFromImage(image, size=size, seed=SEED+i)
        writePointSetGslib(sampled_point_set,
                                     SAMPLES_DIR+'sample_{name}_{size}.gslib'.format(name=name, size=size))

In [ ]:
for reality, name in zip([reality_A, reality_B, reality_C], ['A', 'B', 'C']):
    for i, size, in enumerate(SAMPLE_SIZES):
        sample_and_write_gslib(reality, size, SEED+i, name)


In [ ]:
NX, NY, NZ = 100, 100, 1

def get_and_plot_point_set(filename):
    point_set = readPointSetGslib(filename)
    image = pointSetToImage(point_set, NX, NY, NZ)
    drawImage2D(image, categ=True, categCol=COLOR_SCHEME_BINARY)
    return image

In [ ]:
observations_A = get_and_plot_point_set(SAMPLES_DIR+'sample_A_1600.gslib')

In [ ]:
observations_B = get_and_plot_point_set(SAMPLES_DIR+'sample_B_1600.gslib')

In [ ]:
observations_C = get_and_plot_point_set(SAMPLES_DIR+'sample_C_1600.gslib')

## Roussillon: synthetic test case

In [ ]:
COLOR_SCHEME_ROUSSILLON = [ 
        [x/255 for x in [166,206,227]],
        [x/255 for x in [178,223,138]],   
        [x/255 for x in [31,120,180]],
        [x/255 for x in [51,160,44]],
      ]
LEGEND = ['alluvial fan', 'flood plain', 'splay', 'river bed']

### Training image

In [ ]:
ti_true = readImageGslib(DATA_DIR+'trueTI.gslib')
drawImage2D(ti_true, categ=True, categCol=COLOR_SCHEME_ROUSSILLON, cticklabels=LEGEND)

In [ ]:
drawImage2D(ti_true, iv=1, cmap='cividis', vmin=0.0, vmax=1.0)

In [ ]:
mask = readImageGslib(DATA_DIR+'mask.gslib')
drawImage2D(mask, categ=True, categCol=COLOR_SCHEME_BINARY)

In [ ]:
trend = readImageGslib(DATA_DIR+'trend.gslib')
drawImage2D(trend, cmap='cividis', vmin=0.0, vmax=1.0, excludedVal=-9999999)

In [ ]:
# Orientation
im_angle = readImageGslib(DATA_DIR+'orientation.gslib')
drawImage2D(im_angle, cmap='twilight_shifted', vmin=-180, vmax=180)

### Synthetic reality

In [ ]:
nx, ny, nz = mask.nx, mask.ny, mask.nz      # number of cells
sx, sy, sz = mask.sx, mask.sy, mask.sz      # cell unit
ox, oy, oz = mask.ox, mask.oy, mask.oz      # origin (corner of the "first" grid cell)

deesse_input_roussilon = DeesseInput(
    nx=nx, ny=ny, nz=nz,
    sx=sx, sy=sy, sz=sz,
    ox=ox, oy=oy, oz=oz,
    nv=2, varname=['Facies', 'trend'],
    nTI=1, TI=ti_true,
    mask=mask.val,
    rotationUsage=1,            # use rotation without tolerance
    rotationAzimuthLocal=True,  #    rotation according to azimuth: local
    rotationAzimuth=im_angle.val[0,:,:,:],      #    rotation azimuth: map of values
    dataImage=trend,
    outputVarFlag=[True, False],
    distanceType=[0,1],
    nneighboringNode=[50,1],
    distanceThreshold=[0.05, 0.05],
    maxScanFraction=0.5,
    npostProcessingPathMax=1,
    seed=201912,
    nrealization=1)

In [ ]:
roussillon_simulation = deesseRun(deesse_input=deesse_input_roussilon, nthreads=1)['sim'][0]

### Sample synthetic observation set

In [ ]:
point_set_roussillon_50 = sampleFromImage(roussillon_simulation, size=50, seed=20191201, mask=mask)
point_set_roussillon_150 = sampleFromImage(roussillon_simulation, size=150, seed=20191201, mask=mask)
point_set_roussillon_600 = sampleFromImage(roussillon_simulation, size=600, seed=20191201, mask=mask)

In [ ]:
drawImage2D(roussillon_simulation,  categ=True, categColbad='white', categCol=COLOR_SCHEME_ROUSSILLON, cticklabels=LEGEND)
plt.scatter(point_set_roussillon_50.x(), point_set_roussillon_50.y(), marker= 'x', s=20, c='black')

In [ ]:
drawImage2D(roussillon_simulation,  categ=True, categColbad='white', categCol=COLOR_SCHEME_ROUSSILLON, cticklabels=LEGEND)
plt.scatter(point_set_roussillon_150.x(), point_set_roussillon_150.y(), marker= 'x', s=20, c='black')

In [ ]:
drawImage2D(roussillon_simulation,  categ=True, categColbad='white', categCol=COLOR_SCHEME_ROUSSILLON, cticklabels=LEGEND)
plt.scatter(point_set_roussillon_600.x(), point_set_roussillon_600.y(), marker= 'x', s=10, c='black')

In [ ]:
writePointSetGslib(point_set_roussillon_50, SAMPLES_DIR+'roussillon_observations_50.gslib')
writePointSetGslib(point_set_roussillon_150, SAMPLES_DIR+'roussillon_observations_150.gslib')
writePointSetGslib(point_set_roussillon_600, SAMPLES_DIR+'roussillon_observations_600.gslib')

## Figures for publication

In [ ]:
## Benchmark test case
FONT_SIZE = 16
FIG_DIR = 'figures/'
DPI = 600

import matplotlib

matplotlib.rcParams["image.interpolation"] = None
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

### Training image selection

In [ ]:
def ti_selection_plot(image, filename, title, colorbar=True):
    plt.figure(figsize=(5,6))
    drawImage2D(image,   title=title,
                removeColorbar=not colorbar,
                              categ=True,
                              categColbad='white',
                              categCol=COLOR_SCHEME_BINARY,
                              title_fontsize=FONT_SIZE+2,
                              cticklabels_fontsize=FONT_SIZE,
                              xlabels_fontsize=FONT_SIZE,
                              ylabels_fontsize=FONT_SIZE,
                              xticklabels = [0,100],
                              yticklabels = [0, 100],
                              xticklabels_fontsize=FONT_SIZE,
                              yticklabels_fontsize=FONT_SIZE,
                              xticks=[0,100],
                              yticks=[0,100],
                              ylabel_rotation=0,
                
                             )
    plt.savefig(filename, dpi=DPI)
    #!convert -trim $filename $filename
    !pdfcrop $filename $filename

#### Training images

In [ ]:
ti_selection_plot(ti_A, FIG_DIR+'ti_a.pdf', 'a)', colorbar=False)

In [ ]:
ti_selection_plot(ti_B, FIG_DIR+'ti_b.pdf', 'b)', colorbar=False)

In [ ]:
ti_selection_plot(ti_C, FIG_DIR+'ti_c.pdf', 'c)', colorbar=True)

#### Synthetic realities

In [ ]:
ti_selection_plot(reality_A, FIG_DIR+'reality_a.pdf', 'a)', colorbar=False)

In [ ]:
ti_selection_plot(reality_B, FIG_DIR+'reality_b.pdf', 'b)', colorbar=False)

In [ ]:
ti_selection_plot(reality_C, FIG_DIR+'reality_c.pdf', 'c)', colorbar=True)

#### Observation sets

In [ ]:
ti_selection_plot(observations_A, FIG_DIR+'observations_a.pdf', 'a)', colorbar=False)

In [ ]:
ti_selection_plot(observations_B, FIG_DIR+'observations_b.pdf', 'b)', colorbar=False)

In [ ]:
ti_selection_plot(observations_C, FIG_DIR+'observations_c.pdf', 'c)', colorbar=True)

### Roussillon parameter selection

In [ ]:
EXCLUDED_VAL = -9999999

#### Area with the trend and orientation

In [ ]:
def roussillon_plot(image, filename, title, cmap, vmin, vmax, cticks, cticklabels, colorbar=True, categ=False, iv=0):
    fig = plt.figure(figsize=(5,5))
    fig.subplots_adjust(left=0.05, right=0.9)
    xmin, xmax = [int(x) for x in [image.xmin(), image.xmax()]]
    ymin, ymax = [int(y) for y in [image.ymin(), image.ymax()]]
    drawImage2D(image, iv=iv, excludedVal=EXCLUDED_VAL, cmap=cmap, vmin=vmin, vmax=vmax, title=title,
                removeColorbar=not colorbar,
                categ=categ,
                categCol=COLOR_SCHEME_ROUSSILLON,
                title_fontsize=FONT_SIZE+2,
                cticklabels_fontsize=FONT_SIZE,
                xlabels_fontsize=FONT_SIZE,
                ylabels_fontsize=FONT_SIZE,
                xticklabels = [xmin, xmax],
                yticklabels = [ymin, ymax],
                xticklabels_fontsize=FONT_SIZE,
                yticklabels_fontsize=FONT_SIZE,
                xticks=[xmin, xmax],
                yticks=[ymin, ymax],
                cticks=cticks,
                cticklabels=cticklabels,
                ylabel_rotation=0,
               )
    plt.savefig(filename, dpi=DPI)
    #!convert -trim $filename $filename
    !pdfcrop $filename $filename

In [ ]:
roussillon_plot(trend, filename=FIG_DIR+'trend.pdf', title='a)', cmap='cividis', vmin=0.0, vmax=1.0, cticks=[0.0, 1.0], cticklabels=[0.0, 1.0])

In [ ]:
# In order to limit the orientation over the domain, we must tweak the array values
# We create copy of the orientation and put values outside mask to exclude_value
im_angle_plot = copy.deepcopy(im_angle)

# Iterate over the orientation array and the mask simultaneously and modify in-place
with np.nditer(im_angle_plot.val[0:1,:], op_flags=['readwrite']) as iterator:
    for angle_el, mask_el in zip(iterator, np.nditer(mask.val.flatten())):
        if mask_el == 0:
            angle_el[...] = EXCLUDED_VAL

In [ ]:
roussillon_plot(im_angle_plot, filename=FIG_DIR+'orientation.pdf', title='b)', cmap='twilight_shifted', vmin=-180, vmax=180, cticks=[-180, 0, 180], cticklabels=['$-180^\circ$','$0^\circ$', '$180^\circ$'])

#### Training images

In [ ]:
def roussillon_cat_plot(image, filename, title, colorbar=True):
    fig = plt.figure(figsize=(5,5))
    fig.subplots_adjust(left=0.05, right=0.9)
    xmin, xmax = [int(x) for x in [image.xmin(), image.xmax()]]
    ymin, ymax = [int(y) for y in [image.ymin(), image.ymax()]]
    drawImage2D(image, excludedVal=EXCLUDED_VAL,
                title=title,
                removeColorbar=not colorbar,
                categ=True,
                categCol=COLOR_SCHEME_ROUSSILLON,
                cticklabels=LEGEND,
                title_fontsize=FONT_SIZE+2,
                cticklabels_fontsize=FONT_SIZE,
                xlabels_fontsize=FONT_SIZE,
                ylabels_fontsize=FONT_SIZE,
                xticklabels = [xmin, xmax],
                yticklabels = [ymin, ymax],
                xticklabels_fontsize=FONT_SIZE,
                yticklabels_fontsize=FONT_SIZE,
                xticks=[xmin, xmax],
                yticks=[ymin, ymax],
                ylabel_rotation=0,
               )
    plt.savefig(filename, dpi=DPI, bbox_inches="tight")
    !pdfcrop $filename $filename
    #!convert -trim $filename $filename

In [ ]:
roussillon_cat_plot(ti_true, filename=FIG_DIR+'ti_true.pdf', title='a)')

In [ ]:
roussillon_plot(ti_true, filename=FIG_DIR+'ti_true_trend.pdf', title='b)', cmap='cividis', vmin=0.0, vmax=1.0, cticks=[0.0, 1.0], cticklabels=[0.0, 1.0], iv=1)

In [ ]:
ti_analog = readImageGslib(DATA_DIR+'analogTI.gslib')

In [ ]:
roussillon_cat_plot(ti_analog, filename=FIG_DIR+'ti_analog.pdf', title='a)')

In [ ]:
roussillon_plot(ti_analog, filename=FIG_DIR+'ti_analog_trend.pdf', title='b)', cmap='cividis', vmin=0.0, vmax=1.0, cticks=[0.0, 1.0], cticklabels=[0.0, 1.0], iv=1)

In [ ]:
image = roussillon_simulation

for nsamples, title in zip(['50', '150', '600'], ['a)', 'b)', 'c)']):
    filename = FIG_DIR + "roussillon_observations_{}.pdf".format(nsamples)
    point_set_roussillon = readPointSetGslib(SAMPLES_DIR+"roussillon_observations_{}.gslib".format(nsamples))
    #image = pointSetToImage(point_set_roussillon, image.nx, image.ny, image.nz)
    fig = plt.figure(figsize=(5,5))
    fig.subplots_adjust(left=0.05, right=0.9)
    xmin, xmax = [int(x) for x in [image.xmin(), image.xmax()]]
    ymin, ymax = [int(y) for y in [image.ymin(), image.ymax()]]
    if nsamples == '600':
        # set to False if you want colorbar with 3rd inage
        removeColorbar = True
    else:
        removeColorbar = True
    drawImage2D(image, excludedVal=EXCLUDED_VAL,
                title=title,
                removeColorbar=removeColorbar,
                categ=True,
                categColbad='white',
                categCol=COLOR_SCHEME_ROUSSILLON,
                cticklabels=LEGEND,
                title_fontsize=FONT_SIZE+2,
                cticklabels_fontsize=FONT_SIZE,
                xlabels_fontsize=FONT_SIZE,
                ylabels_fontsize=FONT_SIZE,
                xticklabels = [xmin, xmax],
                yticklabels = [ymin, ymax],
                xticklabels_fontsize=FONT_SIZE,
                yticklabels_fontsize=FONT_SIZE,
                xticks=[xmin, xmax],
                yticks=[ymin, ymax],
                ylabel_rotation=0,
               )
    plt.scatter(point_set_roussillon.x(), point_set_roussillon.y(), marker= 'o', s=5, c='black')
    plt.savefig(filename, dpi=DPI, bbox_inches="tight")
    #!convert -trim $filename $filename
    !pdfcrop $filename $filename